In [1]:
import grpc
import sys
import inference_pb2_grpc as pb2_grpc
import inference_pb2 as pb2

In [124]:
class TestClient(object):
    """
    Client for testing inference gRPC
    """

    def __init__(self):
        self.host = 'localhost'
        self.server_port = 5125

        # instantiate a channel
        self.channel = grpc.insecure_channel(
            '{}:{}'.format(self.host, self.server_port))

        # bind the client and the server
        self.stub = pb2_grpc.InferenceStub(self.channel)

    def send(self, modelHash, modelInput):
        """
        Client function to call the rpc for inference
        """
        message = pb2.InferenceParameters(modelHash=modelHash, modelInput = modelInput)
        print(f'{message}')
        return self.stub.RunInference(message)

def testLinearModel():
    client = TestClient()
    result = client.send(modelHash="LinearTest", modelInput="[[1.0, 2.0, 3.0], [2.5, 3.5, 4.5]]")
    print(f'{result}')

def testVolatilityModel():
    client = TestClient()
    result = client.send(modelHash="Volatility", modelInput="[[0.06219665],[0.02530899],[0.04056685],[0.03235871],[0.05629578]]")
    print(f'{result}')


In [125]:
testLinearModel()

modelHash: "LinearTest"
modelInput: "[[1.0, 2.0, 3.0], [2.5, 3.5, 4.5]]"

value: "[17,26]"



In [126]:
testVolatilityModel()

modelHash: "Volatility"
modelInput: "[[0.06219665],[0.02530899],[0.04056685],[0.03235871],[0.05629578]]"

value: "0.04591319"

